## Imports

In [1]:
import re
import json
from tqdm import tqdm
from bs4 import BeautifulSoup
import bs4
import os
import sys

In [2]:
# Python has a recursion limit of 3K.
# We raise it up to 10K
sys.setrecursionlimit(10000)

In [3]:
DIR_RAW_DATA_PATH = './data/endata/'
DIR_CLEAN_DATA = './data/endata_new_clean/'
JSON_TRAIN_CORPUS = './data/wiki_html_all.json'

## MAIN

In [22]:
# Open Train Corpus file
with open(JSON_TRAIN_CORPUS,'w')as g:
    for root, dirs, files in os.walk(DIR_RAW_DATA_PATH):

        iter_i = 0
        # Iter ZERO
        for dir in dirs:
            #print("DIR: "+dir)
            sub_DIR_RAW_DATA_PATH = os.path.join(root,dir)
            # Get folder
            iter_j=0
            # Iter ONE
            for sub_root,sub_dirs,sub_files in os.walk(sub_DIR_RAW_DATA_PATH):
                if iter_j > 0:
                    extracted_name = sub_root.split("\\")[1].split("(")[0]
                    #print("FOLDER: "+extracted_name)
                iter_j += 1
                # Get file
                iter_k = 0
                # Iter TWO
                for item_parsed in sub_files:
                    path = os.path.join(sub_root,item_parsed)
                    path = re.sub(r'\\', '/', path)
                    clean_path = os.path.join(DIR_CLEAN_DATA,dir+extracted_name+"/clean_"+item_parsed)
                    clean_path = re.sub(r'\((\d+)\)', '-', clean_path)
                    print(f"PATH: {path}")
                    print(f"FILE: {clean_path}")
                    print("")
                    if iter_k == 0:
                        break
                    iter_k += 1

                if iter_j == 2:
                    break
            iter_i =+1 

                

        """ 
        for dir in tqdm(dirs):
            sub_DIR_RAW_DATA_PATH = os.path.join(root,dir)
            for sub_root,sub_dirs,sub_files in os.walk(sub_DIR_RAW_DATA_PATH):
                # foeach file in dir
                for item_parsed in tqdm(sub_files):
                    #index = item_parsed.split('.')[0]
                    path = os.path.join(sub_root,item_parsed)
                    clean_path = DIR_CLEAN_DATA+'_clean.html'
                    cleaner = HTMLCleaner(path,clean_path)
                    cleaner.clean_html()
                    #cleaner.clean_tree()
                    storer = HTMLStorer(input_file=None,html=cleaner.line)
                    g.write(json.dumps(storer.data,ensure_ascii=False)+'\n') """


PATH: ./data/endata/auto/auto-aol(2000)/0000.htm
FILE: ./data/endata_new_clean/autoauto-aol/clean_0000.htm

PATH: ./data/endata/book/book-abebooks(2000)/0000.htm
FILE: ./data/endata_new_clean/bookbook-abebooks/clean_0000.htm

PATH: ./data/endata/camera/camera-amazon(1767)/0000.htm
FILE: ./data/endata_new_clean/cameracamera-amazon/clean_0000.htm

PATH: ./data/endata/job/job-careerbuilder(2000)/0000.htm
FILE: ./data/endata_new_clean/jobjob-careerbuilder/clean_0000.htm

PATH: ./data/endata/movie/movie-allmovie(2000)/0000.htm
FILE: ./data/endata_new_clean/moviemovie-allmovie/clean_0000.htm

PATH: ./data/endata/nbaplayer/nbaplayer-espn(434)/0000.htm
FILE: ./data/endata_new_clean/nbaplayernbaplayer-espn/clean_0000.htm

PATH: ./data/endata/restaurant/restaurant-fodors(2000)/0000.htm
FILE: ./data/endata_new_clean/restaurantrestaurant-fodors/clean_0000.htm

PATH: ./data/endata/university/university-collegeboard(2000)/0000.htm
FILE: ./data/endata_new_clean/universityuniversity-collegeboard/clean

In [31]:
class HTMLCleaner(object):
    def __init__(self, input_file, output_file):
        self.input_file = input_file
        self.output_file = output_file
        
    def clean_tag(self, x):
        # Split the matched tag into components
        x = x.group().split()
        # Check if the first component starts with '<'
        if x[0] == '<':
            # If it does, reconstruct the tag with the first and second components
            y = x[0] + x[1].strip('>') + '>'
        else:
            # If not, reconstruct the tag with the first component
            y = x[0].strip('>') + '>'
        # Return the cleaned tag
        return y
    
    def clean_html(self):
        # Define regular expressions for different HTML elements
        style_regex = "(?:<style.*?>(?:.|[\r\n])*?</style>|<script.*?>(?:.|[\r\n])*?</script>)"
        all_tag_regex = "(?:<(?:!|/?[a-zA-Z]+).*?/?>)"
        close_tag_regex = '(<(?:!|/?[a-zA-Z]+)[^>]*?/>){1}?'
        annotation_regex = '(?:<!--(?:.|[\r\n])*?-->)'
        # Compile regular expressions into patterns
        close_tag_pattern = re.compile(close_tag_regex)
        print(close_tag_pattern)
        annotation_pattern = re.compile(annotation_regex)
        all_tag_pattern = re.compile(all_tag_regex)
        style_pattern = re.compile(style_regex)
        # Initialize lists to store different elements
        tags = []
        annotation = []
        close_tag = []
        all_tag = []
        style = []
        #open file
        with open(self.input_file, 'r') as f:
            line = f.read()
            # Extract annotations
            annotation.extend(annotation_pattern.findall(line))
            line = re.sub(annotation_regex, '', line)
            # Extract close tags
            close_tag.extend(close_tag_pattern.findall(line))
            line = re.sub(close_tag_regex, '', line)
            # Extract styles
            style.extend(style_pattern.findall(line))
            line = re.sub(style_regex, '', line)
            # Extract all tags
            all_tag.extend(all_tag_pattern.findall(line))
            line = re.sub(all_tag_regex, self.clean_tag, line)  # Note: self.clean_tag is missing
        # Store the cleaned line
        self.line = line
    
    def if_text(self,node):
        if node.get_text('|',strip=True) == "":
            return 0
        else:return 1
    
    def get_tree(self,root):
        delete_list = []
        for child in root.children:
            if type(child) == bs4.element.Tag:
                if not self.if_text(child):
                    delete_list.append(child)
                else:
                    child = self.get_tree(child)
            elif type(child) == bs4.element.NavigableString:
                if str(child).strip() == "":
                    delete_list.append(child)
                
        for item in delete_list:
            item.extract()
        return root
    
    def merge_tree(self,root,k):
        i = 0
        while(i<len(root.contents)):
            if(type(root.contents[i])==bs4.element.Tag):
                if len(root.contents[i].contents)<=k:
                    tmp = root.contents[i]
                    j = i
                    del root.contents[i]
                    for item in tmp.contents:
                        root.contents.insert(j,item)
                        j+=1
                else:
                    root.contents[i] = self.merge_tree(root.contents[i],k)
                    i+=1
            else:
                i+=1
        return root
                    
    def clean_tree(self):
        root = BeautifulSoup(self.line,'html.parser').html
        root = self.get_tree(root)
        root = self.merge_tree(root,2)
        self.root = root

    def store(self):
        with open(self.output_file,'w+')as f:
            f.write(str(self.root))

m_path = "./data/endata/auto/auto-aol(2000)/0000.htm"
m_clean_path = "./data/endata_new_clean/autoauto-aol/clean_0000.htm"
cleaner = HTMLCleaner(m_path, m_clean_path)

print(cleaner.get_tree())

TypeError: HTMLCleaner.get_tree() missing 1 required positional argument: 'root'